In [0]:
#@title Imports
import tensorflow as tf
from PIL import Image
import numpy as np
import IPython
from matplotlib import image
from matplotlib import pyplot as plt
import pickle
import scipy.io

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#@markdown Parameters
ngf = 64 # Number of filters in first layer of generator 
ndf = 64 # Number of filters in first layer of discriminator 
stddev = 0.02 # std of Gaussian distribution
batch_size = 1 # batch_size 
pool_size = 50 # pool_size 
img_width = 64 # Imput image will of width 256 
img_height = 64 # Input image will be of height 256 
img_depth = 3 # RGB format
to_restore = False
vgg_layers = scipy.io.loadmat('/content/gdrive/My Drive/MSc ML/0091/vgg19.mat')["layers"]
check_dir = '/content/gdrive/My Drive/MSc ML/0091/64x64/checkpoint/'

In [0]:
def general_conv2d(inputconv, num_features=64, 
                   window_height=7, window_width=7, 
                   stride_height=1, stride_width=1, 
                   padding_size=1, name="conv2d"):
    with tf.variable_scope(name):
        paddings = tf.constant([[0, 0], [padding_size, padding_size,], [padding_size, padding_size], [0, 0]])
        inputconv_ = tf.pad(inputconv, paddings, "REFLECT")
        conv = tf.contrib.layers.conv2d(inputconv_, num_features, [window_height, window_width], 
                                        [stride_height, stride_width], padding='VALID', activation_fn=None, 
                                        weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
                                        biases_initializer=tf.constant_initializer(0.0))
        return conv

      
def general_deconv2d(inputconv, num_features=64, 
                     window_height=3, window_width=3, 
                     stride_height=2, stride_width=2, 
                     padding_size=1, name="deconv2d"):
    with tf.variable_scope(name):
        size = tf.shape(inputconv)[1]
        inputconv = tf.image.resize_nearest_neighbor(inputconv, [size*stride_height,size*stride_width])
        paddings = tf.constant([[0, 0], [padding_size, padding_size,], [padding_size, padding_size], [0, 0]])
        inputconv_ = tf.pad(inputconv, paddings, "REFLECT")
        conv = tf.contrib.layers.conv2d(inputconv_, num_features, [window_height, window_width], 
                                        [1, 1], padding='VALID', activation_fn=None, 
                                        weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
                                        biases_initializer=tf.constant_initializer(0.0))
        return conv

      
def build_resnet_block(input_res, num_features, padding_size=1, name="resnet"):
    with tf.variable_scope(name):

        out_res_1 = general_conv2d(input_res, num_features,
                                   window_width=3, window_height=3,
                                   stride_width=1, stride_height=1, 
                                   padding_size=1, name="c1")
        out_res_2 = general_conv2d(out_res_1, num_features,
                                   window_width=3, window_height=3,
                                   stride_width=1, stride_height=1, 
                                   padding_size=1, name="c2")
        return (out_res_2 + input_res)
      
      
def _weights(layer_idx):
    W = vgg_layers[0][layer_idx][0][0][2][0][0]
    b = vgg_layers[0][layer_idx][0][0][2][0][1]
    return W, b.reshape(b.size)
  
  
def conv2d_relu(prev_layer, layer_idx, layer_name):
    with tf.variable_scope(layer_name):
        # load parameters
        W, b = _weights(layer_idx)
        # initialize parameters
        # W = tf.constant(W, name="weights")
        # b = tf.constant(b, name="bias")
        # convolution 
        conv2d = tf.nn.conv2d(input=prev_layer,
                              filter=W,
                              strides=[1, 1, 1, 1],
                              padding="SAME")
        # activation
        out = tf.nn.relu(conv2d + b)
    return out
    
    
def avgpool(prev_layer, layer_name):
    with tf.variable_scope(layer_name):
        # average pooling
        out = tf.nn.avg_pool(value=prev_layer,
                             ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1],
                             padding="SAME")
    return out
  
  
def gradient_panalty(real, fake, scope="discriminator"):
    alpha = tf.random_uniform(shape=[1], minval=0.0, maxval=1.0)
    interpolated = alpha*real + (1.0 - alpha)*fake
    logit, _ = build_discriminator(interpolated, reuse=tf.AUTO_REUSE, name=scope)
    grad = tf.gradients(logit, interpolated)[0] # gradient of D(interpolated)
    grad_norm = tf.sqrt(tf.reduce_sum(tf.square(grad))) # l2 norm
    GP = tf.reduce_mean(tf.square(grad_norm - 1.0))
    return GP

In [0]:
def build_generator(input_gen, name="generator"):
    with tf.variable_scope(name):
        # input_gen.shape = (1, 64, 64, 3)
    
        # Encoding
        o_c1 = tf.nn.relu(tf.contrib.layers.instance_norm(general_conv2d(input_gen, num_features=ngf, 
                              window_width=7, window_height=7, 
                              stride_width=1, stride_height=1, 
                              padding_size=3, name="c1")))
        # o_c1.shape = (1, 64, 64, 64)
        
        o_c2 = tf.nn.relu(tf.contrib.layers.instance_norm(general_conv2d(o_c1, num_features=ngf*2, 
                              window_width=4, window_height=4, 
                              stride_width=2, stride_height=2, 
                              padding_size=1, name="c2")))
        # o_c2.shape = (1, 32, 32, 128)
        
        o_c3 = tf.nn.relu(tf.contrib.layers.instance_norm(general_conv2d(o_c2, num_features=ngf*4, 
                              window_width=3, window_height=3, 
                              stride_width=2, stride_height=2, 
                              padding_size=1, name="c3")))
        # o_c3.shape = (1, 16, 16, 256)

        # Transformation
        o_r1 = build_resnet_block(o_c3, num_features=ngf*4, padding_size=1, name="r1")
        o_r2 = build_resnet_block(o_r1, num_features=ngf*4, padding_size=1, name="r2")
        o_r3 = build_resnet_block(o_r2, num_features=ngf*4, padding_size=1, name="r3")
        o_r4 = build_resnet_block(o_r3, num_features=ngf*4, padding_size=1, name="r4")
        o_r5 = build_resnet_block(o_r4, num_features=ngf*4, padding_size=1, name="r5")
        o_r6 = build_resnet_block(o_r5, num_features=ngf*4, padding_size=1, name="r6")
        # o_r6.shape = (1, 16, 16, 256)

        #Decoding
        o_d1 = tf.nn.relu(tf.contrib.layers.instance_norm(general_deconv2d(o_r6, num_features=ngf*2, 
                                window_width=3, window_height=3, 
                                stride_width=2, stride_height=2, 
                                padding_size=1, name="d1")))
        # o_d1.shape = (1, 32, 32, 128)
        
        o_d2 = tf.nn.relu(tf.contrib.layers.instance_norm(general_deconv2d(o_d1, num_features=ngf, 
                                window_width=3, window_height=3, 
                                stride_width=2, stride_height=2, 
                                padding_size=1, name="d2")))
        # o_d2.shape = (1, 64, 64, 64)
        
        gen_B = tf.contrib.layers.instance_norm(general_conv2d(o_d2, num_features=3, 
                               window_width=7, window_height=7, 
                               stride_width=1, stride_height=1, 
                               padding_size=3, name="c4"))
        # gen_B.shape = (1, 64, 64, 3)
        
        output = tf.nn.tanh(gen_B, "t1")

        return output

In [0]:
def build_discriminator(input_disc, name="discriminator"):
    with tf.variable_scope(name):

        # input_disc.shape = (1, 64, 64, 3)
        o_c1 = tf.nn.leaky_relu(general_conv2d(input_disc, ndf, 4, 4, 2, 2, padding_size=1, name="c1"), 0.2)
        # o_c1 = tf.nn.dropout(o_c1, rate=0.2)
        # o_c1.shape = (1, 32, 32, 32)
        o_c2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(o_c1, ndf*2, 4, 4, 2, 2, padding_size=1, name="c2"), 0.2))
        # o_c2 = tf.nn.dropout(o_c2, rate=0.2)
        # o_c2.shape = (1, 16, 16, 64)
        o_c3 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(o_c2, ndf*4, 4, 4, 2, 2, padding_size=1, name="c3"), 0.2))
        # o_c3 = tf.nn.dropout(o_c3, rate=0.2)
        # o_c3.shape = (1, 8, 8, 128)
        o_c4 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(o_c3, ndf*8, 4, 4, 2, 2, padding_size=1, name="c4"), 0.2))
        # o_c4 = tf.nn.dropout(o_c4, rate=0.2)
        # o_c4.shape = (1, 4, 4, 256)
        
        decision = tf.sigmoid(general_conv2d(o_c4, 1, 3, 3, 1, 1, padding_size=1, name="c5"))
        # decision.shape = (1, 4, 4, 1)

        return decision

In [0]:
def pac_discriminator(input_disc_1, input_disc_2, name="discriminator"):
    with tf.variable_scope(name):

        input_disc = tf.concat([input_disc_1, input_disc_2], 3)
        # input_disc.shape = (1, 64, 64, 6)
        o_c1 = tf.nn.leaky_relu(general_conv2d(input_disc, ndf, 4, 4, 2, 2, padding_size=1, name="c1"), 0.2)
        # o_c1 = tf.nn.dropout(o_c1, rate=0.2)
        # o_c1.shape = (1, 32, 32, 32)
        o_c2 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(o_c1, ndf*2, 4, 4, 2, 2, padding_size=1, name="c2"), 0.2))
        # o_c2 = tf.nn.dropout(o_c2, rate=0.2)
        # o_c2.shape = (1, 16, 16, 64)
        o_c3 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(o_c2, ndf*4, 4, 4, 2, 2, padding_size=1, name="c3"), 0.2))
        # o_c3 = tf.nn.dropout(o_c3, rate=0.2)
        # o_c3.shape = (1, 8, 8, 128)
        o_c4 = tf.nn.leaky_relu(tf.contrib.layers.instance_norm(general_conv2d(o_c3, ndf*8, 4, 4, 2, 2, padding_size=1, name="c4"), 0.2))
        # o_c4 = tf.nn.dropout(o_c4, rate=0.2)
        # o_c4.shape = (1, 4, 4, 256)
        
        decision = tf.sigmoid(general_conv2d(o_c4, 1, 4, 4, 1, 1, padding_size=2, name="c5"))
        # decision.shape = (1, 4, 4, 1)

        return decision

In [0]:
def content_extractor(input_image, name="extractor"):
    with tf.variable_scope(name):
        conv1_1 = conv2d_relu(input_image, 0, "conv1_1")
        conv1_2 = conv2d_relu(conv1_1, 2, "conv1_2")
        avgpool1 = avgpool(conv1_2, "avgpool1")
        conv2_1 = conv2d_relu(avgpool1, 5, "conv2_1")
        conv2_2 = conv2d_relu(conv2_1, 7, "conv2_2")
        avgpool2 = avgpool(conv2_2, "avgpool2")
        conv3_1 = conv2d_relu(avgpool2, 10, "conv3_1")
        conv3_2 = conv2d_relu(conv3_1, 12, "conv3_2")
        conv3_3 = conv2d_relu(conv3_2, 14, "conv3_3")
        conv3_4 = conv2d_relu(conv3_3, 16, "conv3_4")
        avgpool3 = avgpool(conv3_4, "avgpool3")
        conv4_1 = conv2d_relu(avgpool3, 19, "conv4_1")
        conv4_2 = conv2d_relu(conv4_1, 21, "conv4_2")
        conv4_3 = conv2d_relu(conv4_2, 23, "conv4_3")
        conv4_4 = conv2d_relu(conv4_3, 25, "conv4_4")
        avgpool4 = avgpool(conv4_4, "avgpool4")
        conv5_1 = conv2d_relu(avgpool4, 28, "conv5_1")
        conv5_2 = conv2d_relu(conv5_1, 30, "conv5_2")
        return conv3_2

In [0]:
def style_extractor(input_image, name="extractor"):
        conv1_1 = conv2d_relu(input_image, 0, "conv1_1")
        conv1_2 = conv2d_relu(conv1_1, 2, "conv1_2")
        avgpool1 = avgpool(conv1_2, "avgpool1")
        conv2_1 = conv2d_relu(avgpool1, 5, "conv2_1")
        conv2_2 = conv2d_relu(conv2_1, 7, "conv2_2")
        avgpool2 = avgpool(conv2_2, "avgpool2")
        conv3_1 = conv2d_relu(avgpool2, 10, "conv3_1")
        conv3_2 = conv2d_relu(conv3_1, 12, "conv3_2")
        conv3_3 = conv2d_relu(conv3_2, 14, "conv3_3")
        conv3_4 = conv2d_relu(conv3_3, 16, "conv3_4")
        avgpool3 = avgpool(conv3_4, "avgpool3")
        conv4_1 = conv2d_relu(avgpool3, 19, "conv4_1")
        conv4_2 = conv2d_relu(conv4_1, 21, "conv4_2")
        conv4_3 = conv2d_relu(conv4_2, 23, "conv4_3")
        conv4_4 = conv2d_relu(conv4_3, 25, "conv4_4")
        avgpool4 = avgpool(conv4_4, "avgpool4")
        conv5_1 = conv2d_relu(avgpool4, 28, "conv5_1")

        return conv1_1, conv2_1, conv3_1, conv4_1, conv5_1

In [0]:
'''
def _content_extractor(input_image, name="extractor"):
    with tf.variable_scope(name):
        # input_image.shape = (1, 64, 64, 3)
      
        # window size = 3?
        o_c1 = tf.nn.relu(tf.contrib.layers.instance_norm(general_conv2d(input_image, num_features=64, 
                   window_height=3, window_width=3, 
                   stride_height=1, stride_width=1, 
                   padding_size=1, name="c1")))
        # o_c1.shape = (1, 64, 64, 64)
        
        o_c2 = tf.nn.relu(tf.contrib.layers.instance_norm(general_conv2d(o_c1, num_features=64*2, 
                   window_height=4, window_width=4, 
                   stride_height=2, stride_width=2, 
                   padding_size=1, name="c2")))
        # o_c2.shape = (1, 32, 32, 128)
        
        o_c3 = tf.nn.relu(tf.contrib.layers.instance_norm(general_conv2d(o_c2, num_features=64*2, 
                   window_height=3, window_width=3, 
                   stride_height=1, stride_width=1, 
                   padding_size=1, name="c3")))
        # o_c3.shape = (1, 32, 32, 128)
        
        o_c4 = tf.nn.relu(tf.contrib.layers.instance_norm(general_conv2d(o_c3, num_features=64*4, 
                   window_height=4, window_width=4, 
                   stride_height=2, stride_width=2, 
                   padding_size=1, name="c4")))
        # o_c4.shape = (1, 16, 16, 256)
        
        o_r1 = build_resnet_block(o_c4, num_features=ngf*4, padding_size=1, name="r1")
        # o_r1.shape = (1, 16, 16, 256)
        
        o_c5 = tf.nn.relu(tf.contrib.layers.instance_norm(general_conv2d(o_r1, num_features=64*8, 
                   window_height=4, window_width=4, 
                   stride_height=2, stride_width=2, 
                   padding_size=1, name="c5")))
        # o_c5.shape = (1, 8, 8, 512)
        
        # window size = 3?
        o_c6 = tf.contrib.layers.instance_norm(general_conv2d(o_c5, num_features=64*8, 
                   window_height=3, window_width=3, 
                   stride_height=1, stride_width=1, 
                   padding_size=1, name="c6"))
        # o_c6.shape = (1, 8, 8, 512)
        
        return o_c6
'''

In [0]:
tf.reset_default_graph()

In [0]:
input_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_A")
input_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_B")

buffer_gen_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="buffer_gen_A")
buffer_gen_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="buffer_gen_B")

with tf.variable_scope("Model", reuse=tf.AUTO_REUSE) as scope:
  
    gen_B = build_generator(input_A, name="generator_AtoB")
    gen_A = build_generator(input_B, name="generator_BtoA")

    dec_A = build_discriminator(input_A, name="discriminator_A")
    dec_B = build_discriminator(input_B, name="discriminator_B")
    
    input_A_content = content_extractor(input_A, name="extractor")
    input_B_content = content_extractor(input_B, name="extractor")
    
    input_A_style_1, input_A_style_2, input_A_style_3, input_A_style_4, input_A_style_5 = style_extractor(input_A, name="sextractor")
    input_B_style_1, input_B_style_2, input_B_style_3, input_B_style_4, input_B_style_5 = style_extractor(input_B, name="sextractor")

    dec_gen_A = build_discriminator(gen_A, "discriminator_A")
    dec_gen_B = build_discriminator(gen_B, "discriminator_B")

    cyc_A = build_generator(gen_B, "generator_BtoA")
    cyc_B = build_generator(gen_A, "generator_AtoB")
    
    dec_buffer_gen_A = build_discriminator(buffer_gen_A, "discriminator_A")
    dec_buffer_gen_B = build_discriminator(buffer_gen_B, "discriminator_B")
    
    gen_A_content = content_extractor(gen_A, name="extractor")
    gen_B_content = content_extractor(gen_B, name="extractor")
    
    gen_A_style_1, gen_A_style_2, gen_A_style_3, gen_A_style_4, gen_A_style_5 = style_extractor(gen_A, name="sextractor")
    gen_B_style_1, gen_B_style_2, gen_B_style_3, gen_B_style_4, gen_B_style_5 = style_extractor(gen_B, name="sextractor")

In [0]:
input_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_A")
input_A_2 = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_A_2")
input_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_B")
input_B_2 = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="input_B_2")

buffer_gen_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="buffer_gen_A")
buffer_gen_A_2 = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="buffer_gen_A_2")
buffer_gen_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="buffer_gen_B")
buffer_gen_B_2 = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_depth], name="buffer_gen_B_2")

with tf.variable_scope("Model", reuse=tf.AUTO_REUSE) as scope:
  
    gen_B = build_generator(input_A, name="generator_AtoB")
    gen_B_2 = build_generator(input_A_2, name="generator_AtoB")
    gen_A = build_generator(input_B, name="generator_BtoA")
    gen_A_2 = build_generator(input_B_2, name="generator_BtoA")

    dec_A = pac_discriminator(input_A, input_A_2, name="discriminator_A")
    dec_B = pac_discriminator(input_B, input_B_2, name="discriminator_B")
    
    input_A_content = content_extractor(input_A, name="extractor")
    input_B_content = content_extractor(input_B, name="extractor")
    
    input_A_style_1, input_A_style_2, input_A_style_3, input_A_style_4, input_A_style_5 = style_extractor(input_A, name="sextractor")
    input_B_style_1, input_B_style_2, input_B_style_3, input_B_style_4, input_B_style_5 = style_extractor(input_B, name="sextractor")

    dec_gen_A = pac_discriminator(gen_A, gen_A_2, "discriminator_A")
    dec_gen_B = pac_discriminator(gen_B, gen_B_2, "discriminator_B")

    cyc_A = build_generator(gen_B, "generator_BtoA")
    cyc_B = build_generator(gen_A, "generator_AtoB")
    
    dec_buffer_gen_A = pac_discriminator(buffer_gen_A, buffer_gen_A_2, "discriminator_A")
    dec_buffer_gen_B = pac_discriminator(buffer_gen_B, buffer_gen_B_2, "discriminator_B")
    
    gen_A_content = content_extractor(gen_A, name="extractor")
    gen_B_content = content_extractor(gen_B, name="extractor")
    
    gen_A_style_1, gen_A_style_2, gen_A_style_3, gen_A_style_4, gen_A_style_5 = style_extractor(gen_A, name="sextractor")
    gen_B_style_1, gen_B_style_2, gen_B_style_3, gen_B_style_4, gen_B_style_5 = style_extractor(gen_B, name="sextractor")

In [0]:
# discriminator loss
D_A_loss_1 = tf.reduce_mean(tf.squared_difference(dec_A,1))
D_B_loss_1 = tf.reduce_mean(tf.squared_difference(dec_B,1))

D_A_loss_2 = tf.reduce_mean(tf.square(dec_buffer_gen_A))
D_B_loss_2 = tf.reduce_mean(tf.square(dec_buffer_gen_B))

d_loss_A = (D_A_loss_1 + D_A_loss_2)/2
d_loss_B = (D_B_loss_1 + D_B_loss_2)/2

In [0]:
# cyclic loss (L1)
cyc_loss = tf.reduce_mean(tf.abs(input_A-cyc_A)) + tf.reduce_mean(tf.abs(input_B-cyc_B))

In [0]:
# content loss (L1)
content_loss_BtoA = tf.reduce_mean(tf.abs(input_B_content-gen_A_content))
content_loss_AtoB = tf.reduce_mean(tf.abs(input_A_content-gen_B_content))

In [0]:
# style loss (L1)
style_loss_A = 0.5*tf.reduce_mean(tf.abs(input_A_style_1-gen_A_style_1))+tf.reduce_mean(tf.abs(input_A_style_2-gen_A_style_2))+1.5*tf.reduce_mean(tf.abs(input_A_style_3-gen_A_style_3))+3*tf.reduce_mean(tf.abs(input_A_style_4-gen_A_style_4))+4*tf.reduce_mean(tf.abs(input_A_style_5-gen_A_style_5))
style_loss_B = 0.5*tf.reduce_mean(tf.abs(input_B_style_1-gen_B_style_1))+tf.reduce_mean(tf.abs(input_B_style_2-gen_B_style_2))+1.5*tf.reduce_mean(tf.abs(input_B_style_3-gen_B_style_3))+3*tf.reduce_mean(tf.abs(input_B_style_4-gen_B_style_4))+4*tf.reduce_mean(tf.abs(input_B_style_5-gen_B_style_5))

In [0]:
# variation loss
var_loss_A = tf.image.total_variation(gen_A)
var_loss_B = tf.image.total_variation(gen_B)

In [0]:
# generator loss
g_loss_BtoA = tf.reduce_mean(tf.squared_difference(dec_gen_A,1))
g_loss_AtoB = tf.reduce_mean(tf.squared_difference(dec_gen_B,1))

g_loss_BtoA = g_loss_BtoA + 10*cyc_loss + 5*content_loss_BtoA + 0.0001*var_loss_A + 0.05*style_loss_A
g_loss_AtoB = g_loss_AtoB + 10*cyc_loss + 5*content_loss_AtoB + 0.0001*var_loss_B + 0.05*style_loss_B

In [0]:
# optimizer
lr = tf.placeholder(tf.float32, shape=[], name="lr")

model_vars = tf.trainable_variables()
d_A_vars = [var for var in model_vars if 'discriminator_A' in var.name]
g_AtoB_vars = [var for var in model_vars if 'generator_AtoB' in var.name]
d_B_vars = [var for var in model_vars if 'discriminator_B' in var.name]
g_BtoA_vars = [var for var in model_vars if 'generator_BtoA' in var.name]

optimizer = tf.train.AdamOptimizer(lr, beta1=0.5)
d_A_trainer = optimizer.minimize(d_loss_A, var_list=d_A_vars)
d_B_trainer = optimizer.minimize(d_loss_B, var_list=d_B_vars)
g_AtoB_trainer = optimizer.minimize(g_loss_AtoB, var_list=g_AtoB_vars)
g_BtoA_trainer = optimizer.minimize(g_loss_BtoA, var_list=g_BtoA_vars)

In [0]:
def instance_noise():
    return np.random.uniform(-1.0, 1.0, [1,64,64,3])

In [0]:
with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Painting', 'rb') as f:
    A_input = pickle.load(f)
with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Photo', 'rb') as f:
    B_input = pickle.load(f)    
    
A_input = np.array(A_input)
B_input = np.array(B_input)
A_input = A_input/127.5-1
B_input = B_input/127.5-1

In [0]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    if to_restore:
        # chkpt_fname = tf.train.latest_checkpoint(check_dir)
        saver.restore(sess, '/content/gdrive/My Drive/MSc ML/0091/64x64/outputs/output8/checkpoint/cyclegan-75')

In [0]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    if to_restore:
        # chkpt_fname = tf.train.latest_checkpoint(check_dir)
        saver.restore(sess, '/content/gdrive/My Drive/MSc ML/0091/64x64/checkpoint/model-0')
        with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Buffer/A', 'rb') as f:
            gen_A_buffer = pickle.load(f)
        with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Buffer/B', 'rb') as f:
            gen_B_buffer = pickle.load(f)
        with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Loss/gA2B', 'rb') as f:
            g_loss_AtoB_list = pickle.load(f)
        with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Loss/gB2A', 'rb') as f:
            g_loss_BtoA_list = pickle.load(f)
        with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Loss/dA', 'rb') as f:
            d_loss_A_list = pickle.load(f)
        with open('/content/gdrive/My Drive/MSc ML/0091/64x64/Loss/dB', 'rb') as f:
            d_loss_B_list = pickle.load(f)
    else:
        gen_A_buffer = []
        gen_B_buffer = []
        g_loss_AtoB_list = []
        g_loss_BtoA_list = []
        d_loss_A_list = []
        d_loss_B_list = []
      
    for epoch in range(0, 200):
        print('epoch' + str(epoch))
        # Define the learning rate schedule. The learning rate is kept constant upto 100 epochs and then linearly decayed to 0.
        if(epoch < 100) :
            # learning rate = 0.0002
            curr_lr = 0.0002
        else:
            curr_lr = 0.0002 - 0.0002*(epoch-99)/100
            
        loss_g_A2B = 0 
        loss_g_B2A = 0
        loss_d_A = 0
        loss_d_B = 0
    
        # Running the training loop for all batches
        for ptr in range(0, 2000):
            
            j = np.random.randint(1000)
            m = np.random.randint(1000)
            k = np.random.randint(5000)
            n = np.random.randint(5000)
            
            # Train generator G_A->B
            _, gen_B_temp, g_loss_AtoB_temp = sess.run([g_AtoB_trainer, gen_B, g_loss_AtoB],
                                     feed_dict={input_A:(A_input[j]), input_A_2:(A_input[m]), input_B:(B_input[k]), input_B_2:(B_input[n]), lr:curr_lr})

            loss_g_A2B += g_loss_AtoB_temp/2500
            
            gen_B_buffer.append(gen_B_temp)
            if len(gen_B_buffer) > 50:
                del gen_B_buffer[np.random.randint(50)]

            which = np.random.randint(len(gen_B_buffer))
            B_buffer_gen = gen_B_buffer[which]
            which = np.random.randint(len(gen_B_buffer))
            B_buffer_gen_2 = gen_B_buffer[which]

            # We need gen_B_temp because to calculate the error in training D_B
            _, d_loss_B_temp = sess.run([d_B_trainer, d_loss_B], feed_dict={input_B:((0.5+epoch/400)*B_input[k]+(0.5-epoch/400)*instance_noise()), 
                                                                            input_B_2:((0.5+epoch/400)*B_input[n]+(0.5-epoch/400)*instance_noise()), 
                                                                            buffer_gen_B:(0.5+epoch/400)*B_buffer_gen+(0.5-epoch/400)*instance_noise(), 
                                                                            buffer_gen_B_2:(0.5+epoch/400)*B_buffer_gen_2+(0.5-epoch/400)*instance_noise(), lr:curr_lr})

            loss_d_B += d_loss_B_temp/2000

            # Same for G_B->A and D_A as follow
            _, gen_A_temp, g_loss_BtoA_temp = sess.run([g_BtoA_trainer, gen_A, g_loss_BtoA],
                                     feed_dict={input_A:(A_input[j]), input_A_2:(A_input[m]), input_B:(B_input[k]), input_B_2:(B_input[n]), lr:curr_lr})

            loss_g_B2A += g_loss_BtoA_temp/2000

            gen_A_buffer.append(gen_A_temp)
            if len(gen_A_buffer) > 50:
                del gen_A_buffer[np.random.randint(50)]

            which = np.random.randint(len(gen_A_buffer))
            A_buffer_gen = gen_A_buffer[which]
            which = np.random.randint(len(gen_A_buffer))
            A_buffer_gen_2 = gen_A_buffer[which]

            _, d_loss_A_temp = sess.run([d_A_trainer, d_loss_A], feed_dict={input_A:((0.5+epoch/400)*A_input[j]+(0.5-epoch/400)*instance_noise()), 
                                                                            input_A_2:((0.5+epoch/400)*A_input[m]+(0.5-epoch/400)*instance_noise()), 
                                                                            buffer_gen_A:(0.5+epoch/400)*A_buffer_gen+(0.5-epoch/400)*instance_noise(), 
                                                                            buffer_gen_A_2:(0.5+epoch/400)*A_buffer_gen_2+(0.5-epoch/400)*instance_noise(), lr:curr_lr})
            
            loss_d_A += d_loss_A_temp/2500
            
        g_loss_AtoB_list.append(loss_g_A2B)
        g_loss_BtoA_list.append(loss_g_B2A)
        d_loss_A_list.append(loss_d_A)
        d_loss_B_list.append(loss_d_B)
        print(' loss of generatorAtoB' + str(loss_g_A2B))
        print(' loss of generatorBtoA' + str(loss_g_B2A))
        print(' loss of discriminatorA' + str(loss_d_A))
        print(' loss of discriminatorB' + str(loss_d_B))
            
                
        # save training images
        if epoch%5==0:
          
            saver.save(sess, check_dir + "cyclegan", global_step=epoch)
            for i in range(6):
                gen_A_temp, gen_B_temp, cyc_A_temp, cyc_B_temp = sess.run([gen_A, gen_B, cyc_A, cyc_B], feed_dict={input_A:A_input[i], input_B:B_input[i]})
                Image.fromarray(((gen_A_temp[0]+1)*127.5).astype(np.uint8)).save("/content/gdrive/My Drive/MSc ML/0091/64x64/output9/fakeA_"+ str(i) + "_" + str(epoch), 'JPEG', quality=90)
                Image.fromarray(((gen_B_temp[0]+1)*127.5).astype(np.uint8)).save("/content/gdrive/My Drive/MSc ML/0091/64x64/output9/fakeB_"+ str(i) + "_" + str(epoch), 'JPEG', quality=90)
                Image.fromarray(((cyc_A_temp[0]+1)*127.5).astype(np.uint8)).save("/content/gdrive/My Drive/MSc ML/0091/64x64/output9/cycA_"+ str(i) + "_" + str(epoch), 'JPEG', quality=90)
                Image.fromarray(((cyc_B_temp[0]+1)*127.5).astype(np.uint8)).save("/content/gdrive/My Drive/MSc ML/0091/64x64/output9/cycB_"+ str(i) + "_" + str(epoch), 'JPEG', quality=90)
